# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [ ]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [412]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("../pset3/combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj

id  \
0         None   
1      12-919    
2      11-1002   
3       10-015   
4       18-898   
...        ...   
13082   16-735   
13083   10-473   
13084   17-045   
13085   17-252   
13086   17-304   

                                                                                                                                      title  \
0                                                                                              Convicted Bomb Plotter Sentenced to 30 Years   
1                                                          $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                                             $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                                                      10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4                              $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   
...                                                                                                                                     ...   
13082                        Yuengling to Upgrade Environmental Measures to Settle Clean Water Act Violations at Two Pennsylvania Breweries   
13083                                   Zarein Ahmedzay Pleads Guilty to Terror Violations in Connection with Al-Qaeda New York Subway Plot   
13084                              Zimmer Biomet Holdings Inc. Agrees to Pay $17.4 Million to Resolve Foreign Corrupt Practices Act Charges   
13085  ZTE Corporation Agrees to Plead Guilty and Pay Over $430.4 Million for Violating U.S. Sanctions by Sending U.S.-Origin Items to Iran   
13086                                      ZTE Corporation Pleads Guilty for  Violating U.S. Sanctions by Sending U.S.-Origin Items to Iran   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [413]:
## your code to subset to one press release and take the string
doj_filter = doj[doj['id'] == "17-1204"] 
pharma = ' '.join(doj_filter.contents) # join all to one big string 
type(pharma) # check 


str

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [414]:
## A. 
## your code here to restrict to alpha
pharma_alpha = [word for word in wordpunct_tokenize(pharma) # tokenize and 
                          if word.isalpha()]          # subset for only alphabetical 

pharma_alpha # check 

['The',
 'founder',
 'and',
 'majority',
 'owner',
 'of',
 'Insys',
 'Therapeutics',
 'Inc',
 'was',
 'arrested',
 'today',
 'and',
 'charged',
 'with',
 'leading',
 'a',
 'nationwide',
 'conspiracy',
 'to',
 'profit',
 'by',
 'using',
 'bribes',
 'and',
 'fraud',
 'to',
 'cause',
 'the',
 'illegal',
 'distribution',
 'of',
 'a',
 'Fentanyl',
 'spray',
 'intended',
 'for',
 'cancer',
 'patients',
 'experiencing',
 'breakthrough',
 'pain',
 'More',
 'than',
 'Americans',
 'died',
 'of',
 'synthetic',
 'opioid',
 'overdoses',
 'last',
 'year',
 'and',
 'millions',
 'are',
 'addicted',
 'to',
 'opioids',
 'And',
 'yet',
 'some',
 'medical',
 'professionals',
 'would',
 'rather',
 'take',
 'advantage',
 'of',
 'the',
 'addicts',
 'than',
 'try',
 'to',
 'help',
 'them',
 'said',
 'Attorney',
 'General',
 'Jeff',
 'Sessions',
 'This',
 'Justice',
 'Department',
 'will',
 'not',
 'tolerate',
 'this',
 'We',
 'will',
 'hold',
 'accountable',
 'anyone',
 'from',
 'street',
 'dealers',
 'to',
 

In [415]:
## B 
## your code here for part of speech tagging
pharma_pos = pos_tag(pharma_alpha)
pharma_pos # check 


[('The', 'DT'),
 ('founder', 'NN'),
 ('and', 'CC'),
 ('majority', 'NN'),
 ('owner', 'NN'),
 ('of', 'IN'),
 ('Insys', 'NNP'),
 ('Therapeutics', 'NNP'),
 ('Inc', 'NNP'),
 ('was', 'VBD'),
 ('arrested', 'VBN'),
 ('today', 'NN'),
 ('and', 'CC'),
 ('charged', 'VBN'),
 ('with', 'IN'),
 ('leading', 'VBG'),
 ('a', 'DT'),
 ('nationwide', 'JJ'),
 ('conspiracy', 'NN'),
 ('to', 'TO'),
 ('profit', 'VB'),
 ('by', 'IN'),
 ('using', 'VBG'),
 ('bribes', 'NNS'),
 ('and', 'CC'),
 ('fraud', 'NN'),
 ('to', 'TO'),
 ('cause', 'VB'),
 ('the', 'DT'),
 ('illegal', 'JJ'),
 ('distribution', 'NN'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('Fentanyl', 'NNP'),
 ('spray', 'NN'),
 ('intended', 'VBD'),
 ('for', 'IN'),
 ('cancer', 'NN'),
 ('patients', 'NNS'),
 ('experiencing', 'VBG'),
 ('breakthrough', 'NN'),
 ('pain', 'NN'),
 ('More', 'JJR'),
 ('than', 'IN'),
 ('Americans', 'NNPS'),
 ('died', 'VBD'),
 ('of', 'IN'),
 ('synthetic', 'JJ'),
 ('opioid', 'NN'),
 ('overdoses', 'NNS'),
 ('last', 'JJ'),
 ('year', 'NN'),
 ('and', 'CC'),
 (

In [416]:
## C. 
## Subset for adjectives 
adjectives = [(word, tag) for word, tag in pharma_pos if tag in ('JJ', 'JJR', 'JJS')]
adjectives

[('nationwide', 'JJ'),
 ('illegal', 'JJ'),
 ('More', 'JJR'),
 ('synthetic', 'JJ'),
 ('last', 'JJ'),
 ('medical', 'JJ'),
 ('accountable', 'JJ'),
 ('corporate', 'JJ'),
 ('nationwide', 'JJ'),
 ('American', 'JJ'),
 ('current', 'JJ'),
 ('other', 'JJ'),
 ('former', 'JJ'),
 ('federal', 'JJ'),
 ('later', 'JJ'),
 ('additional', 'JJ'),
 ('several', 'JJ'),
 ('former', 'JJ'),
 ('former', 'JJ'),
 ('former', 'JJ'),
 ('former', 'JJ'),
 ('former', 'JJ'),
 ('former', 'JJ'),
 ('various', 'JJ'),
 ('many', 'JJ'),
 ('powerful', 'JJ'),
 ('narcotic', 'JJ'),
 ('intense', 'JJ'),
 ('large', 'JJ'),
 ('most', 'JJS'),
 ('former', 'JJ'),
 ('reluctant', 'JJ'),
 ('non', 'JJ'),
 ('prior', 'JJ'),
 ('nationwide', 'JJ'),
 ('potent', 'JJ'),
 ('ongoing', 'JJ'),
 ('opioid', 'JJ'),
 ('accountable', 'JJ'),
 ('street', 'JJ'),
 ('level', 'JJ'),
 ('corporate', 'JJ'),
 ('utilized', 'JJ'),
 ('acceptable', 'JJ'),
 ('addictive', 'JJ'),
 ('better', 'JJR'),
 ('street', 'JJR'),
 ('level', 'JJ'),
 ('important', 'JJ'),
 ('pharmaceutical'

## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [417]:
## your code here for part A

doc = nlp(pharma)
entities = [(ent.text, ent.label_) for ent in doc.ents]
entities

[('Insys Therapeutics Inc.', 'ORG'),
 ('today', 'DATE'),
 ('Fentanyl', 'PERSON'),
 ('More than 20,000', 'CARDINAL'),
 ('Americans', 'NORP'),
 ('last year', 'DATE'),
 ('millions', 'CARDINAL'),
 ('Jeff Sessions', 'PERSON'),
 ('This Justice Department', 'ORG'),
 ('Trump', 'PERSON'),
 ('American', 'NORP'),
 ('”John N. Kapoor', 'PERSON'),
 ('74', 'DATE'),
 ('Phoenix', 'GPE'),
 ('Ariz.', 'GPE'),
 ('the Board of Directors', 'ORG'),
 ('Insys', 'ORG'),
 ('this morning', 'TIME'),
 ('Arizona', 'GPE'),
 ('RICO', 'LAW'),
 ('Kapoor', 'PERSON'),
 ('Executive', 'ORG'),
 ('Board', 'ORG'),
 ('Insys', 'ORG'),
 ('Phoenix', 'GPE'),
 ('today', 'DATE'),
 ('U.S.', 'GPE'),
 ('District Court', 'ORG'),
 ('Boston', 'GPE'),
 ('a later date', 'DATE'),
 ('today', 'DATE'),
 ('Boston', 'GPE'),
 ('Insys', 'ORG'),
 ('December 2016.The', 'DATE'),
 ('Kapoor', 'GPE'),
 ('Michael L. Babich', 'PERSON'),
 ('40', 'DATE'),
 ('Scottsdale', 'GPE'),
 ('Ariz.', 'GPE'),
 ('Alec Burlakoff', 'PERSON'),
 ('42', 'DATE'),
 ('Charlotte', 

In [418]:
## your code here for part B
laws = [(word, tag) for word, tag in entities if tag == 'LAW']
laws

[('RICO', 'LAW'), ('the Controlled Substances Act', 'LAW'), ('RICO', 'LAW')]

C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

## Part C- Defining RICO:
A Racketeer Influenced and Corrupt Organizations Act (**RICO**) charge involves the prosecution of peoples involved in organized crime, such as for instance, a pharmaceutical company demonstrating a systamatic pattern of fraudulent, corrupt bribery to increase profits.

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [419]:
## your code here
dates = [(word, tag) for word, tag in entities if tag == 'DATE' and re.search(r'\byear(s)?\b', word, re.IGNORECASE)]
dates

[('last year', 'DATE'),
 ('20 years', 'DATE'),
 ('three years', 'DATE'),
 ('five years', 'DATE'),
 ('three years', 'DATE')]

##### E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [420]:
## convert dates to a list of strings (was tuples, giving issues) 

year_lengths = [year_length for year_length, tag in dates]
year_lengths

## now return all the parts of the press releases where dates are mentioned. 
def contains_year_length(press_release):
    return any(year_length in press_release for year_length in year_lengths)

## subset doj for thoe rows returned by contains_year_length 
filtered_doj = doj[doj['contents'].apply(contains_year_length)]

## Check to make sure above worked, filtered_doj should have less
len(doj)
len(filtered_doj)

## Print rows
(filtered_doj)


['last year', '20 years', 'three years', 'five years', 'three years']

13087

3986

id  \
1      12-919    
15       18-78   
16       18-78   
18      14-550   
19     17-1419   
...        ...   
13063   14-482   
13064   13-678   
13070  10-1127   
13081  14-1377   
13084   17-045   

                                                                                                                                   title  \
1                                                       $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
15                               2 Men Charged With Conspiring to Illegally Obtain Technology and Computer Chips That Were Sent to China   
16                               2 Men Charged With Conspiring to Illegally Obtain Technology and Computer Chips That Were Sent to China   
18                                                                                $20 Million Stolen Identity Refund Fraud Ring Indicted   
19                                        2017 Southeast Regional Animal Cruelty Prosecutions Training Held at Valdosta State University   
...                                                                                                                                  ...   
13063                           Wyoming Businessman Sentenced to Prison for Using Concealed Caribbean Bank Account in Tax Evasion Scheme   
13064                                                                                            Wyoming Couple Indicted for Tax Evasion   
13070                                          Wyoming Used Car Dealer Sentenced to 33 Months in Prison\r\nfor Odometer Tampering Scheme   
13081  Yuba City, California, Man Sentenced to 46 Months in Prison for Racially Motivated Attack on White Man and African-American Woman   
13084                           Zimmer Biomet Holdings Inc. Agrees to Pay $17.4 Million to Resolve Foreign Corrupt Practices Act Charges   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [421]:
## Reg ex pattern to use in seaerch 

pattern = r'(years in)|(years of)'

## Initialize empty list
sentences_with_years = []

## Search each sentence in pharma for the reg ex pattern using re.search & if returned True, add assoc. text to the list initialized above ^^ 
pharma_doc = nlp(pharma)
for sent in pharma_doc.sents:
    if re.search(pattern, sent.text, re.IGNORECASE):
        sentences_with_years.append(sent.text)
sentences_with_years

['The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.\xa0 ',
 'The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.']

The maximum prison length of sentence is **20 years**, with a supervised release set at **3 years**, if the CEO in the Pharma press release is convicted. 

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [422]:
## your code here for subsetting
## reg ex pattern for string begins and ends with one of the three topics 
pattern = r'^(Civil Rights|Hate Crimes|Project Safe Childhood)$'
## subset doj 
doj_subset = doj[doj['topics_clean'].str.contains(pattern)]
## check 
print(f'There are {len(doj_subset)} rows in the subset.')
doj_subset.head()

There are 717 rows in the subset.


,id,title,contents,date,topics_clean,components_clean
77,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle"
155,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to Iraq with the U.S. Army. Evans sexually abused the child on multiple occasions during the 18 months that the child lived with him from May 2007 to December 2008. Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Jacquelyn Hutzell of the Northern District of Alabama are prosecuting the case. U.S. Army Criminal Investigations Division and the FBI’s Birmingham, Alabama, Division investigated the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood, please visit www.justice.gov/psc.",2015-12-11T00:00:00-05:00,Project Safe Childh

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [406]:
## your code here to define function
    
def sentiment_analysis(press_release_string):
    ## process the contents of contents (haha!) with SpaCy
    doc = nlp(press_release_string) 
    
    ## create a reg ex pattern with OR, as reccomended, to look for any and all named entities identified w ent.text 
        ## also used re.escape to handle special characters instead of doing so manually 
    pattern = '|'.join(re.escape(ent.text) for ent in doc.ents if ent.text.strip()) 

    ## use re.sub to as reccomended to replace named entities in press_release_string with an empty string. 
    cleaned_text = re.sub(pattern, '', press_release_string, flags=re.IGNORECASE)

    ## initialize the SentimentIntensity Analyzer 
    sentiment = SentimentIntensityAnalyzer()
    
    ## Score the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
    sentiment_dict = sentiment.polarity_scores(cleaned_text)

    # Return the length-four (negative, positive, neutral, compound) sentiment dictionary 
    return sentiment_dict


In [407]:
## your code here executing the function
for press_release_string in doj_subset.contents:
    print(sentiment_analysis(press_release_string))

{'neg': 0.197, 'neu': 0.754, 'pos': 0.049, 'compound': -0.9931}
{'neg': 0.134, 'neu': 0.797, 'pos': 0.069, 'compound': -0.9325}
{'neg': 0.092, 'neu': 0.832, 'pos': 0.076, 'compound': -0.7579}
{'neg': 0.127, 'neu': 0.788, 'pos': 0.085, 'compound': -0.9037}
{'neg': 0.179, 'neu': 0.777, 'pos': 0.044, 'compound': -0.9864}
{'neg': 0.148, 'neu': 0.799, 'pos': 0.053, 'compound': -0.987}
{'neg': 0.155, 'neu': 0.766, 'pos': 0.079, 'compound': -0.9559}
{'neg': 0.093, 'neu': 0.841, 'pos': 0.066, 'compound': -0.7783}
{'neg': 0.107, 'neu': 0.832, 'pos': 0.061, 'compound': -0.9136}
{'neg': 0.167, 'neu': 0.776, 'pos': 0.056, 'compound': -0.9801}
{'neg': 0.216, 'neu': 0.748, 'pos': 0.036, 'compound': -0.9973}
{'neg': 0.095, 'neu': 0.841, 'pos': 0.064, 'compound': -0.8519}
{'neg': 0.083, 'neu': 0.852, 'pos': 0.065, 'compound': -0.6486}
{'neg': 0.307, 'neu': 0.66, 'pos': 0.034, 'compound': -0.9936}
{'neg': 0.179, 'neu': 0.753, 'pos': 0.069, 'compound': -0.9889}
{'neg': 0.125, 'neu': 0.803, 'pos': 0.071,

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [423]:
## your code here
doj_subset['sentiment_scores'] = doj_subset.contents.apply(sentiment_analysis)
## unpack each dictionary in the column, make each key/category its own column! 
                        ## drop sentiment_scores column keep the rows
                                                                        ## make each key into a column   
doj_subset_wscore = pd.concat([doj_subset.drop(['sentiment_scores'], axis=1), doj_subset['sentiment_scores'].apply(pd.Series)], axis=1)
doj_subset_wscore.columns
doj_subset_wscore.sort_values(by = 'neg', ascending = False) ## put in descending order high to low


Index(['id', 'title', 'contents', 'date', 'topics_clean', 'components_clean',
       'neg', 'neu', 'pos', 'compound'],
      dtype='object')

,id,title,contents,date,topics_clean,components_clean,neg,neu,pos,compound
329,14-248,Albuquerque Man Charged with Federal Hate Crime Related to Anti-Semitic Threats Against Businesswoman,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",2014-03-10T00:00:00-04:00,Hate Crimes,Civil Rights Division; Civil Rights - Criminal Section,0.307,0.660,0.034,-0.9936
572,13-312,Aryan Brother Inmate Sentenced for Federal Hate Crime for Assaulting Fellow Inmate,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",2013-03-14T00:00:00-04:00,Hate Crimes,Civil Rights Division; Civil Rights - 

In [424]:
doj_subset_wscore_C = doj_subset_wscore.head(2) 
doj_subset_wscore_C[['id','contents', 'neg']] 

,id,contents,neg
77,17-1235,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",0.197
155,15-1522,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to Iraq with the U.S. Army. Evans sexually abused the child on multiple occasions during the 18 months that the child lived with him from May 2007 to December 2008. Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Jacquelyn Hutzell of the Northern District of Alabama are prosecuting the case. U.S. Army Criminal Investigations Division and the FBI’s Birmingham, Alabama, Division investigated the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood, please visit www.justice.gov/psc.",0.134


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [425]:
## agg and find the mean compound score by topic
doj_subset_wscore.groupby('topics_clean')['compound'].agg('mean')

topics_clean
Civil Rights             -0.092504
Hate Crimes              -0.935982
Project Safe Childhood   -0.660279
Name: compound, dtype: float64

This variation is likely explained by the social connotations associated with committing a hate crime and exploting children being particularly shameful, given they induce adverse emotional response.

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [426]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [427]:
## your code defining a text processing function
def preprocess_text(text, additional_stopwords):
    
    # Combine custom stopwords with the default stopwords
    default_stopwords = set(stopwords.words('english'))
    all_stopwords = default_stopwords.union(set(additional_stopwords))
    
    # Initialize Snowball stemmer
    stemmer = SnowballStemmer("english")
    
    # Convert contents to lowercase
    text = text.lower()
    
    # Tokenize words in contents 
    words = text.split()
    
    # Remove stopwords, keep alphabetic tokens with length 4 or more
    words = [word for word in words if word not in all_stopwords and word.isalpha() and len(word) >= 4]
    
    # Apply stemming
    words = [stemmer.stem(word) for word in words]
    # Return the processed text
    return ' '.join(words)


In [428]:
## your code executing the function
doj_subset_wscore['processed_text'] = doj_subset_wscore['contents'].apply(lambda x: preprocess_text(x, custom_doj_stopwords))

In [429]:
## your code showing the examples
print(doj_subset_wscore.loc[doj_subset_wscore['id'].isin(['16-718', '16-217']), ['id', 'contents', 'processed_text']])

           id  \
6727   16-217   
11593  16-718   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [432]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(drop=True), dtm_dense_named], axis=1)
    return(dtm_dense_named_withid)

In [567]:
def get_topwords(df, sentiment_type, quantile):
    if sentiment_type == 'positive':
        threshold = df['compound'].quantile(quantile)
        selected_releases = df[df['compound'] > threshold]
        label = ' Most Positive Press Releases'
    elif sentiment_type == 'negative':
        threshold = df['compound'].quantile(quantile)
        selected_releases = df[df['compound'] < threshold]
        label = ' Most Negative Press Releases'
    
    metadata_list = selected_releases[['id', 'compound', 'topics_clean']]
    list_of_strings = selected_releases['processed_text'].tolist()
    
    dtm = create_dtm(list_of_strings, metadata_list)
    display_top_words(dtm, label)

def display_top_words(dtm, subset_label, n=10):
    non_word_columns = ['id', 'compound', 'topics_clean']
    word_columns = dtm.drop(columns=non_word_columns, errors='ignore').select_dtypes(include=[np.number])
    word_sums = word_columns.sum(axis=0)
    sorted_words = word_sums.sort_values(ascending=False)
    top_words = sorted_words.head(n)
    print(f"\nTop 10 words in {subset_label}:")
    print(top_words)
    
# Step B: Top 10 words for press releases with compound sentiment in the top 5%
get_topwords(doj_subset_wscore, 'positive', 0.95)

# Step C: Top 10 words for press releases with compound sentiment in the bottom 5%
get_topwords(doj_subset_wscore, 'negative', 0.05)

# Step D
for topic in dtm['topics_clean'].unique():
    topic_releases = dtm[dtm['topics_clean'] == topic]
    display_top_words(topic_releases, f"press releases for topic '{topic}'")


Top 10 words in  Most Positive Press Releases:
agreement    144
enforc       120
ensur        106
state         94
communiti     84
said          75
servic        73
general       73
provid        72
polic         72
dtype: int64

Top 10 words in  Most Negative Press Releases:
crime       130
offic       121
assault     116
hate        110
defend      109
feder        90
sentenc      88
prosecut     87
victim       84
guilti       83
dtype: int64

Top 10 words in press releases for topic 'Civil Rights':
hous         588
offic        504
enforc       498
said         490
discrimin    476
feder        459
violat       444
alleg        389
general      386
state        347
dtype: int64

Top 10 words in press releases for topic 'Project Safe Childhood':
child       995
exploit     673
sexual      570
safe        475
project     472
crimin      404
prosecut    357
sentenc     329
children    313
investig    256
dtype: int64

Top 10 words in press releases for topic 'Hate Crimes':
prosecut 

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [338]:
# your code here
preprocessed_text = doj_subset_wscore['processed_text']

# Create  model
vectorizer = CountVectorizer(lowercase=True, stop_words='english')
dtm = vectorizer.fit_transform(preprocessed_text)
vocab = vectorizer.get_feature_names_out()

# Convert the DTM into a list of word frequencies
gensim_corpus = [[(i, count) for i, count in enumerate(row) if count > 0] for row in dtm.toarray()]
gensim_dict = corpora.Dictionary.from_corpus(gensim_corpus, id2word=dict(enumerate(vocab)))

# Train an LDA model using Gensim
lda_model = gensim.models.LdaModel(corpus=gensim_corpus, num_topics=3, id2word=gensim_dict, random_state=42)

# Display the top 15 words for each topic
def display_lda_topics(model, num_words=15):
    for topic_id in range(model.num_topics):
        words = model.show_topic(topic_id, num_words)
        print(f"Topic #{topic_id + 1}: " + " ".join([word for word, _ in words]))
display_lda_topics(lda_model)

Topic #1: child hous sexual said discrimin enforc prosecut state general exploit crimin charg inform investig feder
Topic #2: sentenc feder guilti charg child said prosecut investig crime year state defend plead violat offic
Topic #3: prosecut offic general said enforc feder sentenc polic today sexual investig year child charg crimin


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

Part A: Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe. 


In [538]:
## your code here to get doc-level topic probabilities 
document_topics = [lda_model.get_document_topics(doc, minimum_probability = 0) for doc in gensim_corpus] 
document_topics

[[(0, 0.003221889), (1, 0.98277354), (2, 0.014004559)],
 [(0, 0.79061824), (1, 0.0043349667), (2, 0.2050468)],
 [(0, 0.7499364), (1, 0.05501781), (2, 0.19504581)],
 [(0, 0.71746784), (1, 0.045845125), (2, 0.23668705)],
 [(0, 0.0036483894), (1, 0.0037361847), (2, 0.99261546)],
 [(0, 0.0029602465), (1, 0.9939866), (2, 0.0030531965)],
 [(0, 0.854145), (1, 0.003907867), (2, 0.14194717)],
 [(0, 0.3369616), (1, 0.36639237), (2, 0.29664603)],
 [(0, 0.003451244), (1, 0.0034336322), (2, 0.9931151)],
 [(0, 0.72897476), (1, 0.0038276436), (2, 0.2671976)],
 [(0, 0.0025305126), (1, 0.99483454), (2, 0.0026349542)],
 [(0, 0.1109064), (1, 0.002595601), (2, 0.886498)],
 [(0, 0.7708085), (1, 0.0027782617), (2, 0.2264132)],
 [(0, 0.10493142), (1, 0.33293962), (2, 0.56212896)],
 [(0, 0.0035699822), (1, 0.4761567), (2, 0.5202733)],
 [(0, 0.3607696), (1, 0.6366765), (2, 0.0025539245)],
 [(0, 0.8358622), (1, 0.16209182), (2, 0.002046015)],
 [(0, 0.0021882257), (1, 0.98356825), (2, 0.014243571)],
 [(0, 0.0025

Part B: Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

In [539]:
## your code here to add those topic probabilities to the dataframe
highest_probability_topics = [max(topic_list, key=lambda x: x[1] )[0] + 1 for topic_list in document_topics]
# highest_probability_topics
doj_subset_wscore['top_topics'] = highest_probability_topics 
doj_subset_wscore['top_topics']

77       2
155      1
157      1
162      1
168      3
        ..
13002    2
13032    1
13034    1
13068    1
13081    2
Name: top_topics, Length: 717, dtype: int64

Part C: For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

In [540]:
doj_crosstab = pd.crosstab(doj_subset_wscore['topics_clean'], doj_subset_wscore['top_topics'], normalize='index') * 100
doj_crosstab

top_topics,1,2,3
topics_clean,,,
Civil Rights,46.229508,27.213115,26.557377
Hate Crimes,6.097561,67.073171,26.829268
Project Safe Childhood,46.385542,31.325301,22.289157


Part D: Using a couple of press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)

In [541]:
cr_examples = doj_subset_wscore[doj_subset_wscore['topics_clean'] == 'Civil Rights'].head(1)
hc_examples = doj_subset_wscore[doj_subset_wscore['topics_clean'] == 'Hate Crimes'].head(1)
cr_examples.contents
hc_examples.contents

77    A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened.     James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack.   Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up.   “Every citizen has the right to due process and protection from unreasonable force, and correc

168    Jeremy Heath Higgins was indicted for threatening an African-American man at a Quinton, Alabama, restaurant, and for threatening another person who ordered Higgins to leave the restaurant due to his behavior, Acting Assistant Attorney General Jocelyn Samuels for the Justice Department’s Civil Rights Division and U.S. Attorney Joyce Vance for the Northern District of Alabama announced today.   Higgins, 28, was charged in a three count indictment returned yesterday by a federal grand jury in the U.S. District Court for the Northern District of Alabama.  The indictment charges him with one felony count and two misdemeanor counts of interference with a federally-protected activity.  The indictment alleges that on June 14, 2013, Higgins approached and threatened an African-American man at the Alabama Rose Steakhouse because the man was present at the restaurant with a white woman.  According to the indictment, another person ordered Higgins to leave the premises of the restaurant bec

Hate Crimes very strongly maps unto topic 2, because it contains broad legal terminology typical to hate crime cases.
Civil Rights maps most strongly to topic 1, because they contain language typical of civil rights-related issues and child-issues such as discrim, hous, child. 


# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [549]:
def create_bigram_onedoc(text):
    words = text.split()
    bigrams = ['_'.join(pair) for pair in zip(words, words[1:])]
    return ' '.join(bigrams)

doj_subset_wscore['processed_text_bigrams'] = doj_subset_wscore['processed_text'].apply(create_bigram_onedoc)

(doj_subset_wscore.loc[doj_subset_wscore['id'] == '16-217', ['id', 'processed_text', 'processed_text_bigrams']])


,id,processed_text,processed_text_bigrams
6727,16-217,reach comprehens settlement agreement citi miami miami polic resolv shoot announc princip deputi general vanita head wifredo ferrer southern approv citi commiss today effect agreement sign resolv claim stem shoot conduct violent crime control enforc issu juli identifi pattern practic excess forc shoot violat fourth amend complianc settlement monitor independ former polic chief jane settlement citi implement comprehens reform ensur constitut polic support public settlement agreement design minim shoot effect quick investig shoot measur settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain said princip deputi general agreement help strengthen relationship communiti serv improv account offic fire weapon provid communiti particip enforc agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut said oversight agreement seek make perman posit chang former chief orosa chief llane applaud citi settlement agreement build upon import reform implement citi sinc issu conduct attorney staff special litig section southern,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_shoot shoot_announc announc_princip princip_deputi deputi_general general_vanita vanita_head head_wifredo wifredo_ferrer ferrer_southern southern_approv approv_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_resolv resolv_claim claim_stem stem_shoot shoot_conduct conduct_violent violent_crime crime_control control_enforc enforc_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_shoot shoot_violat violat_fourth fourth_amend amend_complianc complianc_settlement settlement_monitor monitor_independ independ_former former_polic polic_chief chief_jane jane_settlement settlement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_settlement settlement_agreement agreement_design design_minim minim_shoot shoot_effect effect_quick quick_investig investig_shoot shoot_measur measur_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_said said_princip princip_deputi deputi_general general_agreement agreement_help help_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_provid provid_communiti communiti_particip particip_enforc enforc_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_communiti communiti_safe safe_protect protect_sacr sacr_constitut constitut_said said_oversight oversight_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief chief_orosa orosa_chief chief_llane llane_applaud applaud_citi citi_settlement settlement_agreement agreement_build build_upon upon_import import_reform reform_implement implement_citi citi_sinc sinc_issu issu_conduct conduct_attorney attorney_staff staff_special special_litig litig_section section_southern


C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [559]:
# your code here
metadata_list = doj_subset_wscore[['id','topics_clean','compound']]
processed_text_bigrams = doj_subset_wscore['processed_text_bigrams']
dtm_bigram = create_dtm(processed_text_bigrams, metadata_list) 

x = dtm.shape
y = dtm_bigram.shape 

print(f'The dimensions of the dtm matrix from question 2.2 are {x}, while the dimensions of the dtm_bigram matrix are {y}.')


The dimensions of the dtm matrix from question 2.2 are (717, 6110), while the dimensions of the dtm_bigram matrix are (717, 63644)


The reason why we see so many more dimensions in the bigram_dtm than the dtm from 2.2 is because the 2.2 dtm's features are unigrams. The bigram_dtm has many more features, because each individual word forms a pair with the next, with each pair being an individual feature; this is why we see so many more features in the bigram_dtm. 

In [568]:
for topic in dtm_bigram['topics_clean'].unique():
    topic_releases = dtm_bigram[dtm_bigram['topics_clean'] == topic]
    display_top_words(topic_releases, f"press releases for topic '{topic}'")


Top 10 words in press releases for topic 'Civil Rights':
fair_hous         225
deputi_general    221
princip_deputi    221
vanita_head       200
general_vanita    199
said_princip      186
announc_today     116
unit_state        106
consent_decre      92
act_general        80
dtype: int64

Top 10 words in press releases for topic 'Project Safe Childhood':
project_safe         472
child_exploit        266
child_pornographi    237
sexual_exploit       218
plead_guilti         188
child_sexual         177
exploit_obscen       175
safe_childhood       161
obscen_section       161
individu_exploit     156
dtype: int64

Top 10 words in press releases for topic 'Hate Crimes':
hate_crime       299
plead_guilti     269
special_agent    117
year_prison      104
thoma_general     95
grand_juri        93
said_thoma        91
act_general       85
face_maximum      77
feder_hate        76
dtype: int64


# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [ ]:
# your code here 